In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
import h5py
import os
import argparse
import multiprocessing as mp
import yaml
import warnings
# import queue
from queue import LifoQueue, Queue, Empty
from threading import Thread

In [2]:
def initialize_hdf5(filename, framesize=None, codec=None):
    filename = filename + '.h5'
    f = h5py.File(filename, 'w')
    datatype = h5py.special_dtype(vlen=np.dtype('uint8'))
    dset = f.create_dataset('frame', (0,), maxshape=(None,),dtype=datatype)
    # dset = f.create_dataset('right', (0,), maxshape=(None,),dtype=datatype)
    return(f)

def write_frame_hdf5(writer_obj, frame, axis=0):
    # ret1, left_jpg = cv2.imencode('.jpg', left, (cv2.IMWRITE_JPEG_QUALITY,80))
    # ret2, right_jpg = cv2.imencode('.jpg', right, (cv2.IMWRITE_JPEG_QUALITY,80))
    ret, jpg = cv2.imencode('.jpg', frame, (cv2.IMWRITE_JPEG_QUALITY,80))
    f['frame'].resize(f['frame'].shape[axis]+1, axis=axis)
    # f['left'].resize(f['left'].shape[axis]+1, axis=axis)
    f['frame'][-1]=jpg.squeeze()
     
def initialize_opencv(filename, framesize, codec):
    filename = filename + '.avi'
    fourcc = cv2.VideoWriter_fourcc(*codec)
    writer = cv2.VideoWriter(filename,fourcc, 30, framesize)
    return(writer)
def write_frame_opencv(writer_obj, frame):
    # out = cv2.cvtColor(np.hstack((left, right)), cv2.COLOR_GRAY2RGB)
    writer_obj.write(frame)
    
def initialize_ffmpeg(filename,framesize, codec=None):
    filename = filename + '.avi'
    size_string = '%dx%d' %framesize
    outname = os.path.join(outdir, fname)
    command = [ 'ffmpeg',
        '-threads', '1',
        '-y', # (optional) overwrite output file if it exists
        '-f', 'rawvideo',
        '-vcodec','rawvideo',
        '-s', size_string, # size of one frame
        '-pix_fmt', 'rgb24',
        '-r', '30', # frames per second
        '-i', '-', # The imput comes from a pipe
        '-an', # Tells FFMPEG not to expect any audio
        '-vcodec', 'libx264',
        '-crf', '23', 
        outname]
    pipe = sp.Popen( command, stdin=sp.PIPE, stderr=sp.STDOUT)
    return(pipe)
# from here 
# https://zulko.github.io/blog/2013/09/27/read-and-write-video-frames-in-python-using-ffmpeg/
def write_frame_ffmpeg(pipe, frame):
    # out = cv2.cvtColor(np.hstack((left,right)), cv2.COLOR_GRAY2RGB)
    try:
        pipe.stdin.write(frame.tobytes())
    except BaseException as err:
        _, ffmpeg_error = pipe.communicate()
        error = (str(err) + ("\n\nerror: FFMPEG encountered "
                             "the following error while writing file:"
                             "\n\n %s" % (str(ffmpeg_error))))
        
def append_to_hdf5(f, name, value, axis=0):
    f[name].resize(f[name].shape[axis]+1, axis=axis)
    f[name][-1]=value

In [3]:
class Device:
    def __init__(self, start_t=None,height=None,width=None,save=False,savedir=None,
                 experiment=None, name=None,
        movie_format='hdf5',metadata_format='hdf5', preview=False,verbose=False,options=None):
        # print('Initializing %s' %name)
        self.config = config
        self.serial = serial
        self.start_t = start_t
        self.save= save
        self.savedir = savedir
        self.experiment = experiment
        self.name = name
        # self.save_format=save_format
        self.preview=preview
        self.verbose = verbose
        self.options = options
        self.started = False
        self.height = height
        self.width = width
        
        assert(movie_format in ['hdf5', 'opencv', 'ffmpeg'])
        assert(metadata_format in ['hdf5', 'csv'])
        self.movie_format = movie_format
        self.metadata_format = metadata_format
        
        if movie_format == 'hdf5':
            self.initialization_func == initialize_hdf5
            self.write_frame = write_frame_hdf5
        elif movie_format == 'opencv':
            self.initialization_func = initialize_opencv
            self.write_frame = write_frame_opencv
        elif movie_format == 'ffmpeg':
            self.initialization_func = initialize_ffmpeg
            self.write_frame = write_frame_ffmpeg
    
    def process(self):
        # should be overridden by all subclasses
        raise NotImplementedError    

    def start(self):
        # should be overridden by all subclasses
        raise NotImplementedError

    def initialize_preview(self):
        # cv2.namedWindow(self.name, cv2.WINDOW_NORMAL)
        cv2.namedWindow(self.name, cv2.WINDOW_AUTOSIZE)
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.latest_frame = None
        self.preview_queue = LifoQueue(maxsize=5)
        self.preview_thread = Thread(target=self.preview_worker, args=(self.preview_queue,))
        self.preview_thread.daemon = True
        self.preview_thread.start()

    def preview_worker(self, queue):
        should_continue = True
        while should_continue:
            item = queue.get()
            # print(item)
            if item is None:
                if self.verbose:
                    print('Preview stop signal received')
                should_continue=False
                break
                # break
            # left, right, count = item
            frame, count = item
            # frame should be processed, so a single RGB image
            # out = np.vstack((left,right))
            h, w, c = frame.shape
            if self.save:
                frame = cv2.resize(frame, (w//2,h//2),cv2.INTER_NEAREST)
                out_height = h//2
            else:
                frame = cv2.resize(frame, (w,h),cv2.INTER_NEAREST)
                out_height = h//3*2
            # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
                    
            # string = '%.4f' %(time_acq*1000)
            string = '%s:%07d' %(self.name, count)
            cv2.putText(frame,string,(10,out_height-20), self.font, 0.5,(0,0,255),2,cv2.LINE_AA)
            self.latest_frame = frame

            queue.task_done()
        
    
    def initialize_metadata_saving_hdf5(self):
        fname = os.path.join(self.directory, self.name + '_metadata.h5')
        f = h5py.File(fname, 'w')
        
        dset = f.create_dataset('framecount',(0,),maxshape=(None,),dtype=np.int32)
        dset = f.create_dataset('timestamp',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('arrival_time',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('sestime',(0,),maxshape=(None,),dtype=np.float64)
        dset = f.create_dataset('cputime',(0,),maxshape=(None,),dtype=np.float64)
        
        self.metadata_obj = f
        
    def write_metadata(self, framecount, timestamp, arrival_time, sestime, cputime):
        append_to_hdf5(self.metadata_obj,'framecount', framecount)
        append_to_hdf5(self.metadata_obj,'timestamp', timestamp)
        append_to_hdf5(self.metadata_obj,'arrival_time', arrival_time)
        append_to_hdf5(self.metadata_obj,'sestime', sestime)
        append_to_hdf5(self.metadata_obj, 'cputime', cputime)
        
    def save_worker(self, queue):
        should_continue = True
        while should_continue:
            try:
                item = queue.get()
                # print(item)
                if item is None:
                    if self.verbose:
                        print('Saver stop signal received')
                    should_continue = False
                    break
                # left, right, count = item
                frame, metadata = item

                self.write_frame(self.writer_obj, frame)
                self.write_metadata(*metadata)

                
                print(queue.qsize())
                time.sleep(1/120)
            except Exception as e:
                print(e)
            finally:
                queue.task_done()
        
        print('out of save queue')
        
    def initialize_saving(self):
        assert(self.savedir is not None and self.experiment is not None)
        directory = os.path.join(self.savedir, self.experiment)
        if not os.path.isdir(directory):
            os.makedirs(directory)
        self.directory = directory
        
        if self.metadata_format == 'hdf5':
            self.initialize_metadata_saving_hdf5()
        else:
            raise NotImplementedError
        
        framesize = (self.width, self.height)
        codec = 'DIVX'
        filename = os.path.join(self.directory, self.name)
        writer_obj = self.initialization_func(filename, framesize, codec)
        self.writer_obj = writer_obj
        
        self.save_queue = Queue(maxsize=600)
        self.save_thread = Thread(target=self.save_worker, args=(self.save_queue,))
        self.save_thread.daemon = True
        self.save_thread.start()

    def update_settings(self, sync_mode='master'):
        # should be overridden by subclass
        raise NotImplementedError

    def stop_streaming(self):
        # should be overridden by subclass
        # print(dir(self.pipeline))
        raise NotImplementedError

    def stop(self):
        # if self.preview:
        if not self.started:
            return
        if self.save:
            print('Waiting for saving thread to finish on cam %s. DO NOT INTERRUPT' %self.name)
            self.save_queue.put(None)
            print('joining...')
            self.save_queue.join()
            print('joined')
        if self.preview:
            self.preview_queue.put(None)
            self.preview_thread.join()
            cv2.destroyWindow(self.name)
        if hasattr(self, 'pipeline'):
            # print('stream')
            self.stop_streaming()
        if hasattr(self, 'writer_obj'):
            # print('videoobj')
            self.writer_obj.release()
            # del(self.videoobj)
        if hasattr(self, 'metadata_obj'):
            # print('fileobj')
            self.metadata_obj.close()
        
        self.started = False
        print('Cam %s stopped' %self.name) 

    def __del__(self):
        try:
            self.stop()
        except BaseException as e:
            if self.verbose:
                print('Error in destructor of cam %s' %self.name)
                print(e)
            else:
                pass

In [4]:
class Realsense(Device):
    def __init__(self,serial, config,
                 start_t=None,height=None,width=None,save=False,savedir=None,experiment=None, name=None,
        movie_format='hdf5',metadata_format='hdf5', preview=False,verbose=False,options=None):
        super().__init__(start_t,height, width, save, savedir, experiment, name, 
                        movie_format, metadata_format, preview, verbose, options)
        self.serial = serial
        self.config = config
        
    def process(self, left, right):
        t0 = time.perf_counter()
        out = np.hstack((left, right))
        out = cv2.cvtColor(out, cv2.COLOR_GRAY2RGB)
        print('process t: %.6f' %( (time.perf_counter() - t0)*1000 ))
        return(out)
    
    def start(self, sync_mode='slave'):
        pipeline = rs.pipeline()
        self.config.enable_device(self.serial)
        try:
            pipeline_profile = pipeline.start(self.config)
        except RuntimeError:
            print('Pipeline for camera %s already running, restarting...' %self.serial)
            pipeline.stop()
            time.sleep(1)
            pipeline_profile = pipeline.start(self.config)
        CAPACITY = 10
        # print(dir(pipeline))
        # self.framequeue = rs.framequeue(CAPACITY)
        self.pipeline = pipeline
        self.prof = pipeline_profile
        time.sleep(1)
        self.update_settings(sync_mode)
        if self.save:
            self.initialize_saving()
            print('saving initialized: %s' %self.name)
        if self.preview:
            self.initialize_preview()
        self.started= True
        
    def update_settings(self, sync_mode='master'):
        # sensor = self.prof.get_device().first_depth_sensor()
        # print(dir(sensor))
        # sensor.set_option(rs.option.emitter_enabled,1)
        if sync_mode=='master':
            mode = 1
        elif sync_mode == 'slave':
            mode = 2
        if self.verbose:
            print('%s: %s,%d' %(self.name, sync_mode, mode))
        
        this_device = self.prof.get_device()
        ir_sensors = this_device.query_sensors()[0] # 1 for RGB
        if self.options=='default' or self.options=='large':
            ir_sensors.set_option(rs.option.emitter_enabled,1)
            ir_sensors.set_option(rs.option.enable_auto_exposure,0)
            laser_pwr = ir_sensors.get_option(rs.option.laser_power)
            if self.verbose:
                print("laser power = ", laser_pwr)
            laser_range = ir_sensors.get_option_range(rs.option.laser_power)
            if self.verbose:
                print("laser power range = " , laser_range.min , "~", laser_range.max)
            ir_sensors.set_option(rs.option.laser_power,300)
            ir_sensors.set_option(rs.option.exposure,650)
            ir_sensors.set_option(rs.option.gain,16)

        elif self.options =='calib':
            ir_sensors.set_option(rs.option.emitter_enabled,0)
            ir_sensors.set_option(rs.option.enable_auto_exposure,0)
            ir_sensors.set_option(rs.option.exposure,1200)
            ir_sensors.set_option(rs.option.gain,16)
            self.jpg_quality = 99

        if self.options=='large':
            ir_sensors.set_option(rs.option.exposure,750)
            ir_sensors.set_option(rs.option.laser_power,200)
            ir_sensors.set_option(rs.option.gain,16)
        if self.options=='brighter':
            gain_range = ir_sensors.get_option_range(rs.option.gain)
            if self.verbose:
                print("gain range = " , gain_range.min , "~", gain_range.max)
            ir_sensors.set_option(rs.option.exposure, 500)
            ir_sensors.set_option(rs.option.gain,16)
        # set this to 2 for slave mode, 1 for master!
        ir_sensors.set_option(rs.option.inter_cam_sync_mode, mode)
        
    def stop_streaming(self):
        # print(dir(self.pipeline))
        try:
            self.pipeline.stop()
            self.config.disable_all_streams()
        except BaseException as e:
            if self.verbose:
                print('Probably tried to call stop before a start.')
                print(e)
            else:
                pass

In [5]:
if os.path.isfile('serials.yaml'):
    with open('serials.yaml') as f:
        serial_dict = yaml.load(f, Loader=yaml.SafeLoader)
else:
    warnings.Warn('Need to create a lookup table for matching serial numbers with names.')
    serial_dict = {}
    for serial in serials:
        serial_dict[serial] = None

In [6]:
serial = '830112071475'
start_t = time.perf_counter()
options = 'large'
save = True
preview = True
verbose=True

datadir = r'D:\DATA\JB\realsense'
experiment = 'testing_inheritance_%s' %time.strftime('%y%m%d_%H%M%S', time.localtime())
# name = serial_dict

In [7]:
config = rs.config()
if options=='default' or options=='brighter':
    resolution_width = 480
    resolution_height = 270
    framerate = 90
elif options=='large':
    resolution_width = 640
    resolution_height = 480
    framerate=60
elif options=='calib':
    resolution_width=640
    resolution_height=480
    framerate=6
else:
    raise NotImplementedError
config.enable_stream(rs.stream.infrared, 1, resolution_width, resolution_height, rs.format.y8, framerate)
config.enable_stream(rs.stream.infrared, 2, resolution_width, resolution_height, rs.format.y8, framerate)
device = Realsense(serial, config, start_t,resolution_height, resolution_width*2, save=save,
                   savedir=datadir, experiment=experiment,
        name=serial_dict[serial],preview=preview,verbose=verbose, options=options,
        movie_format='opencv')

In [8]:
device.stop()

In [9]:
device.start()

right_post: slave,2
laser power =  200.0
laser power range =  0.0 ~ 360.0
saving initialized: right_post


In [10]:
try:
    should_continue = True
    while should_continue:
        # print('acquiring')
        frames = device.pipeline.wait_for_frames(1000*10)
        # frames =  rs.composite_frame(rs.frame())
        # frames = framequeue.wait_for_frame(1000*10)
        # frame = framequeue.poll_for_frame()
        # frames = framequeue.poll_for_frame()
        # print(frames)
        # if frames:
            # print(dir(frames))
        start_t = time.perf_counter()
        # frames = device.pipeline.poll_for_frames()
        left = frames.get_infrared_frame(1)
        right = frames.get_infrared_frame(2)
        if not left or not right:
            continue
        left, right = np.asanyarray(left.get_data()), np.asanyarray(right.get_data())
        frame = device.process(left, right)
        sestime = time.perf_counter() - device.start_t
        cputime = time.time()
        framecount = frames.get_frame_number()
        # by default, milliseconds from 1970. convert to seconds for datetime.datetime
        arrival_time = frames.get_frame_metadata(rs.frame_metadata_value.time_of_arrival)/1000
        timestamp = frames.get_timestamp()/1000
        
        # def write_metadata(self, framecount, timestamp, arrival_time, sestime, cputime):
        metadata = (framecount, timestamp, arrival_time, sestime, cputime)
        if device.save:
            device.save_queue.put_nowait((frame, metadata))
            #             device.write_frame(frame, framecount, timestamp,
            #                 arrival_time, sestime, cputime)
            # if saving, be more stringent about previewing
            # condition = (time.perf_counter()-start_t)*1000<8 and framecount%5==0
          

        # print(time.perf_counter()-start_t)
        if device.preview and framecount % 10 ==0:
            # print(time.perf_counter()-start_t)
            device.preview_queue.put((frame,framecount))
            if device.latest_frame is not None:
                cv2.imshow(device.name, device.latest_frame)
                key = cv2.waitKey(1)
                if key==27:
                    break
        
except KeyboardInterrupt:
    print('keyboard interrupt')
    should_continue = False
    # print('User stopped acquisition.')
finally:
    # don't know why I can't put this in the destructor
    # print(dir(device))
    # if device.preview:
    #     device.preview_queue.put(None)
    #     device.preview_thread.join()
    # time.sleep(1)
    device.stop()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
17
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
45
46
47
49
51
52
53
54
55
56
57
58
59
61
62
64
66
67
68
69
71
73
74
75
76
77
78
79
80
81
83
85
87
89
90
91
93
95
96
97
99
101
103
104
105
106
107
108
109
110
111
112
113
114
116
117
118
120
122
124
125
126
127
127
129
130
131
132
133
135
137
138
139
140
142
144
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
73
75
76
77
79
80
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
115
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
17

Full: 